In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import re
import sys
import os
from torchvision.transforms import v2
import torch
import shutil
from PIL import Image
from ultralytics import YOLO
import cv2
import pandas as pd
from PIL import Image
import random
import albumentations as A
from albumentations.pytorch import ToTensorV2
import numpy as np
from IPython.display import clear_output
import yaml
import os
from PIL import Image



# Local dep
project_dir = '/data/konrad/workspace'
sys.path.insert(0, project_dir)

from helpers.datasets import CalfCenterFaceDataset
from helpers.helpers import get_indices, uniform_sample_with_values, load_face_data

INFO:albumentations.check_version:A new version of Albumentations is available: 1.4.14 (you have 1.4.12). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.


In [2]:
def plot_bbox_gallery(image_df, n_cols=5):
    n_images = image_df.shape[0]
    n_rows = n_images // n_cols + int(n_images % n_cols > 0)
    
    plt.figure(figsize=(20, n_rows * 4))
    for i, row in image_df.iterrows():
        img = Image.open(row["path"])
        # plt.subplot(n_rows, n_cols, i + 1)
        ax = plt.subplot(n_rows, n_cols, i + 1)
        plt.imshow(img)
        # ax = plt.gca()
        score = row["conf"]
        box_x = row["box_x"]
        box_y = row["box_y"]
        box_width = row["box_width"]
        box_height = row["box_height"]
        box_x = box_x - (box_width / 2)
        box_y = box_y - (box_height / 2)
        rect = plt.Rectangle((box_x, box_y), box_width, box_height, edgecolor='r', facecolor='none')
        ax.add_patch(rect)
        plt.text(box_x, box_y - 10, f' {score:.2f}', color='red', fontsize=12, backgroundcolor='white')
        plt.axis('off')
    
    plt.tight_layout()
    plt.show()


def apply_transformations(image, bbox, transforms, label_id):
    transformed = transforms(image=image, bboxes=[bbox], category_ids=[label_id])
    
    transformed_image = transformed['image']
    transformed_bbox = transformed['bboxes'][0]
    
    return transformed_image, transformed_bbox
    
def df_to_yolo(df, root_dir, label_col = "label", transforms = None, num_gen=3):
    
    img_path = root_dir + "/"
    if not os.path.exists(img_path):
        os.makedirs(img_path)

    for index, row in df.iterrows():
        # Extract the image name
        image_label = row[label_col]
        
        lbl_path = img_path + "/" + image_label
        if not os.path.exists(lbl_path):
            os.makedirs(lbl_path)
        
        image_name = row['image']

        img_src_dir = row['path']
        
        class_idx = row['target']

        xmin = row['x_min']
        ymin = row['y_min']
        xmax = row['x_max']
        ymax = row['y_max']

        bbox = [xmin, ymin, xmax, ymax]

        if transforms is None:

            filename = os.path.splitext(image_name)[0]

            # Open the image file
            with Image.open(os.path.join(img_src_dir)) as img:
            # Get the width and height
                img_pil = img.crop((xmin, ymin, xmax, ymax))
                img_pil.save(os.path.join(lbl_path, image_name))

            continue

        img = cv2.imread(img_src_dir)
        
        for i in range(num_gen):  # Apply transformations 5 times
            transformed_image, transformed_bbox = apply_transformations(img, bbox, transforms, class_idx)
            transformed_image_name = f"{os.path.splitext(image_name)[0]}_transformed_{i}.jpg"
            
            xmin, ymin, xmax, ymax = transformed_bbox
            img_rgb = cv2.cvtColor(transformed_image, cv2.COLOR_BGR2RGB)
            img_pil = Image.fromarray(img_rgb).crop((xmin, ymin, xmax, ymax))
            img_pil.save(os.path.join(lbl_path, transformed_image_name))

    print("Done !")


def delete_dir_if_exists(dir_path):
    if os.path.exists(dir_path):
        try:
            shutil.rmtree(dir_path)
            print(f"Directory '{dir_path}' deleted")
        except OSError as e:
            print(f"Error deleting directory '{dir_path}': {e}")

def set_yml(file_name, data_path, id2label, train_dir = "train", test_dir = "test", val_dir = "val"):
    
    # Create the dictionary structure for YAML
    yolo_detect_config = {
        'path': data_path,
        'train': train_dir,
        'test': test_dir,
        'val': val_dir,
        'names': id2label
    }
    
    # Define the output file path
    yml_file_path = os.path.join(data_path, f"{file_name}.yml")
    
    # Write the YAML file
    with open(yml_file_path, 'w') as yml_file:
        yaml.dump(yolo_detect_config, yml_file, default_flow_style=False)
        
    print("Yml file set !")
    return yml_file_path

In [3]:
ROOT_DIR = "/data/konrad/workspace"

dataset_type = "splt"
label_col = "label"

num_labels = 3 if label_col == "label" else 1
train_df = pd.read_csv(ROOT_DIR + f'/csv_files/mixed_10s_b0s_y7_1/{dataset_type}_train_image_extracted_metadata.csv', index_col=False)
valid_df = pd.read_csv(ROOT_DIR + f'/csv_files/mixed_10s_b0s_y7_1/{dataset_type}_test_image_extracted_metadata.csv', index_col=False)
test_df = pd.read_csv(ROOT_DIR + f'/csv_files/mixed_10s_b0s_y7_1/{dataset_type}_val_image_extracted_metadata.csv', index_col=False)
labels = train_df[label_col].unique()
label2id = {l:i for i, l in enumerate(labels)}
id2label = {i:l for i, l in enumerate(labels)}
train_df['target'] = train_df.apply(lambda row: label2id[row[label_col]], axis=1)
test_df['target'] = test_df.apply(lambda row: label2id[row[label_col]], axis=1)
valid_df['target'] = valid_df.apply(lambda row: label2id[row[label_col]], axis=1)
# test_df = test_df.sample(frac=.10).reset_index(drop=True)

IMAGE_DIR = ROOT_DIR + f"/datasets/yolo_classify_{dataset_type}"
num_gen = 1

In [5]:
transforms = A.Compose([
    A.Rotate(limit=(10, 20), p=.7),
    A.OneOf([
        A.GaussianBlur(blur_limit=(3,9), p=.7),
        A.MedianBlur(blur_limit=7, p=.7),
    ], p=.6),
    A.Sharpen(p=.7),
], bbox_params=A.BboxParams(format='pascal_voc', label_fields=['category_ids']))


delete_dir_if_exists(IMAGE_DIR)
df_to_yolo(train_df, IMAGE_DIR + '/train', transforms = transforms, num_gen=num_gen)
# df_to_yolo(train_df, IMAGE_DIR + '/train', img_src_dir = IMAGE_DIR, transforms = None, num_gen=num_gen)
df_to_yolo(valid_df, IMAGE_DIR + '/val')
df_to_yolo(test_df, IMAGE_DIR + '/test')

# Load a model
model = YOLO("yolov8m-cls.pt")  # load a pretrained model (recommended for training)
epoch = 10

model_name = "yolo_classifier"
# yml_file_path = set_yml(f"{model_name}_{dataset_type}", IMAGE_DIR, id2label)
results = model.train(data=IMAGE_DIR, epochs=epoch, project=ROOT_DIR + "/training_log", name=model_name, cfg="config.yml")
# results = model.train(data="yolo_detect.yml", epochs=epoch, project=ROOT_DIR + "/training_log", name="yolo_detect")
# results = model.val(data=IMAGE_DIR, split='test')  # Evaluate on test set

model.export()

Directory '/data/konrad/workspace/datasets/yolo_classify_splt' deleted
Done !
Done !
Done !
New https://pypi.org/project/ultralytics/8.2.84 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.58 🚀 Python-3.10.12 torch-2.3.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4090, 24115MiB)
engine/trainer: task=classify, mode=train, model=yolov8m-cls.pt, data=/data/konrad/workspace/datasets/yolo_classify_splt, epochs=10, time=None, patience=100, batch=16, imgsz=224, save=True, save_period=-1, cache=False, device=None, workers=8, project=/data/konrad/workspace/training_log, name=yolo_classifier6, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False

train: Scanning /data/konrad/workspace/datasets/yolo_classify_splt/train... 5497 images, 0 corrupt: 100%|██████████| 549

train: New cache created: /data/konrad/workspace/datasets/yolo_classify_splt/train.cache



val: Scanning /data/konrad/workspace/datasets/yolo_classify_splt/val... 283 images, 0 corrupt: 100%|██████████| 283/283 

val: New cache created: /data/konrad/workspace/datasets/yolo_classify_splt/val.cache


...

                   all      0.378          1



10 epochs completed in 0.025 hours.
Optimizer stripped from /data/konrad/workspace/training_log/yolo_classifier6/weights/last.pt, 31.7MB
Optimizer stripped from /data/konrad/workspace/training_log/yolo_classifier6/weights/best.pt, 31.7MB

Validating /data/konrad/workspace/training_log/yolo_classifier6/weights/best.pt...
Ultralytics YOLOv8.2.58 🚀 Python-3.10.12 torch-2.3.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4090, 24115MiB)
YOLOv8m-cls summary (fused): 103 layers, 15,766,499 parameters, 0 gradients, 41.6 GFLOPs
train: /data/konrad/workspace/datasets/yolo_classify_splt/train... found 5497 images in 3 classes ✅ 
val: /data/konrad/workspace/datasets/yolo_classify_splt/val... found 283 images in 3 classes ✅ 
test: /data/konrad/workspace/datasets/yolo_classify_splt/test... found 1379 images in 3 classes ✅ 


               classes   top1_acc   top5_acc: 100%|██████████| 9/9 [00:00<00:00, 22.38it/s]


                   all      0.452          1
Speed: 0.2ms preprocess, 0.6ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to /data/konrad/workspace/training_log/yolo_classifier6
Results saved to /data/konrad/workspace/training_log/yolo_classifier6
Ultralytics YOLOv8.2.58 🚀 Python-3.10.12 torch-2.3.1+cu121 CPU (AMD Ryzen Threadripper PRO 5975WX 32-Cores)
YOLOv8m-cls summary (fused): 103 layers, 15,766,499 parameters, 0 gradients, 41.6 GFLOPs

PyTorch: starting from '/data/konrad/workspace/training_log/yolo_classifier6/weights/best.pt' with input shape (1, 3, 224, 224) BCHW and output shape(s) (1, 3) (30.2 MB)

TorchScript: starting export with torch 2.3.1+cu121...
TorchScript: export success ✅ 0.5s, saved as '/data/konrad/workspace/training_log/yolo_classifier6/weights/best.torchscript' (60.4 MB)

Export complete (1.8s)
Results saved to /data/konrad/workspace/training_log/yolo_classifier6/weights
Predict:         yolo predict task=classify model=/data/konrad/workspace

'/data/konrad/workspace/training_log/yolo_classifier6/weights/best.torchscript'

In [4]:
transforms = A.Compose([
    A.Rotate(limit=(10, 20), p=.7),
    A.OneOf([
        A.GaussianBlur(blur_limit=(3,9), p=.7),
        A.MedianBlur(blur_limit=7, p=.7),
    ], p=.6),
    A.Sharpen(p=.7),
], bbox_params=A.BboxParams(format='pascal_voc', label_fields=['category_ids']))

delete_dir_if_exists(IMAGE_DIR)
# df_to_yolo(train_df, IMAGE_DIR + '/train', img_src_dir = IMAGE_DIR, transforms = transforms, num_gen=num_gen)
df_to_yolo(train_df, IMAGE_DIR + '/train', transforms = None, num_gen=num_gen)
df_to_yolo(valid_df, IMAGE_DIR + '/test')
df_to_yolo(test_df, IMAGE_DIR + '/val')

# Load a model
model = YOLO("yolov8m-cls.pt")  # load a pretrained model (recommended for training)
epoch = 10

model_name = "yolo_classifier"
# yml_file_path = set_yml(f"{model_name}_{dataset_type}", IMAGE_DIR, id2label)
# results = model.train(data=yml_file_path, epochs=epoch, project=ROOT_DIR + "/training_log", name="yolo_detect", cfg="config.yml")
results = model.train(data=IMAGE_DIR, epochs=epoch, project=ROOT_DIR + "/training_log", name=model_name)
results = model.val(data=IMAGE_DIR, split='test')  # Evaluate on test set

model.export()

Directory '/data/konrad/workspace/datasets/yolo_classify_splt' deleted
Done !
Done !
Done !
New https://pypi.org/project/ultralytics/8.2.83 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.58 🚀 Python-3.10.12 torch-2.3.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4090, 24115MiB)
engine/trainer: task=classify, mode=train, model=yolov8m-cls.pt, data=/data/konrad/workspace/datasets/yolo_classify_splt, epochs=10, time=None, patience=100, batch=16, imgsz=224, save=True, save_period=-1, cache=False, device=None, workers=8, project=/data/konrad/workspace/training_log, name=yolo_classifier4, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False

train: Scanning /data/konrad/workspace/datasets/yolo_classify_splt/train... 5497 images, 0 corrupt: 100%|██████████| 549


train: New cache created: /data/konrad/workspace/datasets/yolo_classify_splt/train.cache


val: Scanning /data/konrad/workspace/datasets/yolo_classify_splt/val... 1379 images, 0 corrupt: 100%|██████████| 1379/13


val: New cache created: /data/konrad/workspace/datasets/yolo_classify_splt/val.cache
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000714, momentum=0.9) with parameter groups 38 weight(decay=0.0), 39 weight(decay=0.0005), 39 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 224 train, 224 val
Using 8 dataloader workers
Logging results to /data/konrad/workspace/training_log/yolo_classifier4
Starting training for 10 epochs...

      Epoch    GPU_mem       loss  Instances       Size


...

test: Scanning /data/konrad/workspace/datasets/yolo_classify_splt/test... 283 images, 0 corrupt: 100%|██████████| 283/28

test: New cache created: /data/konrad/workspace/datasets/yolo_classify_splt/test.cache



               classes   top1_acc   top5_acc: 100%|██████████| 18/18 [00:00<00:00, 30.56it/s]


                   all      0.452          1
Speed: 0.2ms preprocess, 1.5ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to /data/konrad/workspace/training_log/yolo_classifier42
Ultralytics YOLOv8.2.58 🚀 Python-3.10.12 torch-2.3.1+cu121 CPU (AMD Ryzen Threadripper PRO 5975WX 32-Cores)

PyTorch: starting from '/data/konrad/workspace/training_log/yolo_classifier4/weights/best.pt' with input shape (1, 3, 224, 224) BCHW and output shape(s) (1, 3) (30.2 MB)

TorchScript: starting export with torch 2.3.1+cu121...
TorchScript: export success ✅ 0.6s, saved as '/data/konrad/workspace/training_log/yolo_classifier4/weights/best.torchscript' (60.4 MB)

Export complete (1.8s)
Results saved to /data/konrad/workspace/training_log/yolo_classifier4/weights
Predict:         yolo predict task=classify model=/data/konrad/workspace/training_log/yolo_classifier4/weights/best.torchscript imgsz=224  
Validate:        yolo val task=classify model=/data/konrad/workspace/training_log/yolo_cla

'/data/konrad/workspace/training_log/yolo_classifier4/weights/best.torchscript'

In [6]:
transforms = A.Compose([
    A.Rotate(limit=(10, 20), p=.7),
    A.OneOf([
        A.GaussianBlur(blur_limit=(3,9), p=.7),
        A.MedianBlur(blur_limit=7, p=.7),
    ], p=.6),
    A.Sharpen(p=.7),
], bbox_params=A.BboxParams(format='pascal_voc', label_fields=['category_ids']))


delete_dir_if_exists(IMAGE_DIR)
df_to_yolo(train_df, IMAGE_DIR + '/train', transforms = transforms, num_gen=num_gen)
# df_to_yolo(train_df, IMAGE_DIR + '/train', img_src_dir = IMAGE_DIR, transforms = None, num_gen=num_gen)
df_to_yolo(valid_df, IMAGE_DIR + '/test')
df_to_yolo(test_df, IMAGE_DIR + '/val')

# Load a model
model = YOLO("yolov8m-cls.pt")  # load a pretrained model (recommended for training)
epoch = 10

model_name = "yolo_classifier"
# yml_file_path = set_yml(f"{model_name}_{dataset_type}", IMAGE_DIR, id2label)
results = model.train(data=IMAGE_DIR, epochs=epoch, project=ROOT_DIR + "/training_log", name=model_name, cfg="config.yml")
# results = model.train(data="yolo_detect.yml", epochs=epoch, project=ROOT_DIR + "/training_log", name="yolo_detect")
results = model.val(data=IMAGE_DIR, split='test')  # Evaluate on test set

model.export()

Directory '/data/konrad/workspace/datasets/yolo_classify_splt' deleted
Done !
Done !
Done !
New https://pypi.org/project/ultralytics/8.2.83 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.58 🚀 Python-3.10.12 torch-2.3.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4090, 24115MiB)
engine/trainer: task=classify, mode=train, model=yolov8m-cls.pt, data=/data/konrad/workspace/datasets/yolo_classify_splt, epochs=10, time=None, patience=100, batch=16, imgsz=224, save=True, save_period=-1, cache=False, device=None, workers=8, project=/data/konrad/workspace/training_log, name=yolo_classifier5, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False

train: Scanning /data/konrad/workspace/datasets/yolo_classify_splt/train... 5497 images, 0 corrupt: 100%|██████████| 549

train: New cache created: /data/konrad/workspace/datasets/yolo_classify_splt/train.cache



val: Scanning /data/konrad/workspace/datasets/yolo_classify_splt/val... 1379 images, 0 corrupt: 100%|██████████| 1379/13

val: New cache created: /data/konrad/workspace/datasets/yolo_classify_splt/val.cache


...

test: Scanning /data/konrad/workspace/datasets/yolo_classify_splt/test... 283 images, 0 corrupt: 100%|██████████| 283/28

test: New cache created: /data/konrad/workspace/datasets/yolo_classify_splt/test.cache



               classes   top1_acc   top5_acc: 100%|██████████| 18/18 [00:00<00:00, 42.76it/s]


                   all       0.42          1
Speed: 0.3ms preprocess, 0.7ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to /data/konrad/workspace/training_log/yolo_classifier52
Ultralytics YOLOv8.2.58 🚀 Python-3.10.12 torch-2.3.1+cu121 CPU (AMD Ryzen Threadripper PRO 5975WX 32-Cores)

PyTorch: starting from '/data/konrad/workspace/training_log/yolo_classifier5/weights/best.pt' with input shape (1, 3, 224, 224) BCHW and output shape(s) (1, 3) (30.2 MB)

TorchScript: starting export with torch 2.3.1+cu121...
TorchScript: export success ✅ 0.5s, saved as '/data/konrad/workspace/training_log/yolo_classifier5/weights/best.torchscript' (60.4 MB)

Export complete (1.7s)
Results saved to /data/konrad/workspace/training_log/yolo_classifier5/weights
Predict:         yolo predict task=classify model=/data/konrad/workspace/training_log/yolo_classifier5/weights/best.torchscript imgsz=224  
Validate:        yolo val task=classify model=/data/konrad/workspace/training_log/yolo_cla

'/data/konrad/workspace/training_log/yolo_classifier5/weights/best.torchscript'